# Warp factory for Python

This code represents the outcome of an exploration I conducted on how to compute the stress-energy tensor for a given spacetime metric. My goal is to replicate the functionality of the "Warp Factory" code described in the paper arXiv:2404.03095.

In contrast to the approach in that study, I aim to leverage the full capabilities of SymPy and EinsteinPy to derive analytical expressions for various quantities of interest, avoiding reliance on numerical approximations whenever possible. This symbolic approach allows for greater flexibility and precision in theoretical investigations.

I may introduce additional constraints on the physicality of the energy-momentum tensor, particularly when negative energy arises, as discussed in arXiv
/9702026.

At this stage, the code is more of a collection of potential applications without a clear overarching theme or direction.

### import relevant library

In [1]:
import sympy as sp
from einsteinpy.symbolic import MetricTensor, ChristoffelSymbols, RiemannCurvatureTensor, RicciTensor, RicciScalar, EinsteinTensor
from sympy import init_printing

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [43]:

#PARTIAL DERIVATIVES

# Step 1: Define coordinates (t, x, y, z)
t, x, y, z = sp.symbols('t x y z')
symbols = [t, x, y, z]

# Step 2: Define the vector V^v in terms of coordinates
V_0 = sp.Function('V_0')(t, x, y, z)
V_1 = sp.Function('V_1')(t, x, y, z)
V_2 = sp.Function('V_2')(t, x, y, z)
V_3 = sp.Function('V_3')(t, x, y, z)

V = [V_0, V_1, V_2, V_3]

# Step 3: Initialize an empty matrix for partial derivatives
partial_matrix = sp.Matrix.zeros(4, 4)

# Step 4: Compute the partial derivatives for all combinations
for i in range(4):  # iterate over coordinates u^i
    for j in range(4):  # iterate over vector components V^v
        partial_matrix[i, j] = sp.diff(V[j], symbols[i])

display(partial_matrix)




Matrix([
[Derivative(V_0(t, x, y, z), t), Derivative(V_1(t, x, y, z), t), Derivative(V_2(t, x, y, z), t), Derivative(V_3(t, x, y, z), t)],
[Derivative(V_0(t, x, y, z), x), Derivative(V_1(t, x, y, z), x), Derivative(V_2(t, x, y, z), x), Derivative(V_3(t, x, y, z), x)],
[Derivative(V_0(t, x, y, z), y), Derivative(V_1(t, x, y, z), y), Derivative(V_2(t, x, y, z), y), Derivative(V_3(t, x, y, z), y)],
[Derivative(V_0(t, x, y, z), z), Derivative(V_1(t, x, y, z), z), Derivative(V_2(t, x, y, z), z), Derivative(V_3(t, x, y, z), z)]])

In [44]:
# Define the symbols for coordinates and vector components
t, x, y, z = sp.symbols('t x y z')
V_h = sp.symbols('V_h0 V_h1 V_h2 V_h3')  # Vector V_h (covariant components)
V_up_h = sp.symbols('V^h0 V^h1 V^h2 V^h3')  # Vector V^h (contravariant components)

# Define a simple metric (for example, flat spacetime)
eta = sp.diag(-1, 1, 1, 1)  # Minkowski metric (flat spacetime)



# Compute Christoffel Symbols for the given metric
ch_sym = ChristoffelSymbols.from_metric(m_obj)

# Christoffel symbol array (with lower indices)
T_uv_h = ch_sym.tensor()

display(sp.simplify(T_uv_h))


# Initialize matrices A and B
A_uv = sp.zeros(4, 4)  # A_{uv} matrix (4x4)
B_u_v = sp.zeros(4, 4)  # B_u^v matrix (4x4)

# Perform the matrix multiplication for A_uv = T_{uv}^h * V_h
for u in range(4):
    for v in range(4):
        A_uv[u, v] = sum(T_uv_h[h,u,v] * V_h[h] for h in range(4))

# Perform the matrix multiplication for B_u^v = T_{uh}^v * V^h
for u in range(4):
    for v in range(4):
        B_u_v[u, v] = sum(T_uv_h[v,u,h]  * V_up_h[h] for h in range(4))

display(A_uv)
display(B_u_v)

[[[f(t, x, y, z)**2*v(t)**3*Derivative(f(t, x, y, z), x), -f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x), -f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), y)/2, -f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), z)/2], [-f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x), v(t)*Derivative(f(t, x, y, z), x), v(t)*Derivative(f(t, x, y, z), y)/2, v(t)*Derivative(f(t, x, y, z), z)/2], [-f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), y)/2, v(t)*Derivative(f(t, x, y, z), y)/2, 0, 0], [-f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), z)/2, v(t)*Derivative(f(t, x, y, z), z)/2, 0, 0]], [[f(t, x, y, z)**3*v(t)**4*Derivative(f(t, x, y, z), x) - f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x) - f(t, x, y, z)*Derivative(v(t), t) - v(t)*Derivative(f(t, x, y, z), t), -f(t, x, y, z)**2*v(t)**3*Derivative(f(t, x, y, z), x), (-f(t, x, y, z)**2*v(t)**2 - 1)*v(t)*Derivative(f(t, x, y, z), y)/2, (-f(t, x, y, z)**2*v(t)**2 - 1)*v(t)*Derivative(f(t, x, y, z), z)/2], [-f(t, x, y, z)**2*v(t)**3*Derivativ

Matrix([
[V_h0*(-(-2*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x) - 2*f(t, x, y, z)*Derivative(v(t), t) - 2*v(t)*Derivative(f(t, x, y, z), t))*f(t, x, y, z)*v(t)/2 - f(t, x, y, z)**2*v(t)*Derivative(v(t), t) - f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), t)) + V_h1*((-f(t, x, y, z)**2*v(t)**2/2 + 1/2)*(-2*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x) - 2*f(t, x, y, z)*Derivative(v(t), t) - 2*v(t)*Derivative(f(t, x, y, z), t)) - (2*f(t, x, y, z)**2*v(t)*Derivative(v(t), t) + 2*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), t))*f(t, x, y, z)*v(t)/2) - V_h2*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), y) - V_h3*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), z), -V_h0*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x) - V_h1*f(t, x, y, z)**2*v(t)**3*Derivative(f(t, x, y, z), x) + V_h2*v(t)*Derivative(f(t, x, y, z), y)/2 + V_h3*v(t)*Derivative(f(t, x, y, z), z)/2, -V_h0*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), y)/2 + V_h1*(-(-f(t, x, y, z)**2*v(t)**2/2 + 1/2)*v(t)*

Matrix([
[V^h0*(-(-2*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x) - 2*f(t, x, y, z)*Derivative(v(t), t) - 2*v(t)*Derivative(f(t, x, y, z), t))*f(t, x, y, z)*v(t)/2 - f(t, x, y, z)**2*v(t)*Derivative(v(t), t) - f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), t)) - V^h1*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x) - V^h2*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), y)/2 - V^h3*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), z)/2, V^h0*((-f(t, x, y, z)**2*v(t)**2/2 + 1/2)*(-2*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x) - 2*f(t, x, y, z)*Derivative(v(t), t) - 2*v(t)*Derivative(f(t, x, y, z), t)) - (2*f(t, x, y, z)**2*v(t)*Derivative(v(t), t) + 2*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), t))*f(t, x, y, z)*v(t)/2) - V^h1*f(t, x, y, z)**2*v(t)**3*Derivative(f(t, x, y, z), x) + V^h2*(-(-f(t, x, y, z)**2*v(t)**2/2 + 1/2)*v(t)*Derivative(f(t, x, y, z), y) - f(t, x, y, z)**2*v(t)**3*Derivative(f(t, x, y, z), y)) + V^h3*(-(-f(t, x, y, z)**2*v(t)**2/2 + 1/2)*v(t)*Derivat

In [45]:
def christoffel_x_vector(V,metric_einsteinpy,sp_symbols, vector_is_covariant):
    # The given metric must be a einsteinpy object not a tensor!

    dim=len(sp_symbols)

    # Compute Christoffel Symbols for the given metric
    ch_sym = ChristoffelSymbols.from_metric(metric_einsteinpy)
    
    # Christoffel symbol array (with lower indices)
    T_uv_h = ch_sym.tensor()



    if vector_is_covariant:
        # Initialize matrices A and B
        A_uv = sp.zeros(dim, dim)  # A_{uv} matrix (4x4)
        
        # Perform the matrix multiplication for A_uv = T_{uv}^h * V_h
        for u in range(dim):
            for v in range(dim):
                A_uv[u, v] = sum(T_uv_h[h,u,v] * V[h] for h in range(dim)) #first index is the upper index for christoffel symbol


        return A_uv
        
    else:
        # Initialize matrices A and B
        B_u_v = sp.zeros(dim, dim)  # B_u^v matrix (4x4)
        
        
        # Perform the matrix multiplication for B_u^v = T_{uh}^v * V^h
        for u in range(dim):
            for v in range(dim):
                B_u_v[u, v] = sum(T_uv_h[v,u,h]  * V[h] for h in range(dim))

        return B_u_v




In [46]:
def covariant_derivative(V,metric_einsteinpy,sp_symbols, vector_is_covariant):

    dim=len(sp_symbols)

    #Initialize an empty matrix for partial derivatives
    partial_matrix = sp.Matrix.zeros(dim,dim)
    
    #Compute the partial derivatives for all combinations
    for i in range(dim):  # iterate over coordinates u^i
        for j in range(dim):  # iterate over vector components V^v
            partial_matrix[i, j] = sp.diff(V[j], sp_symbols[i])
    

    
    ch_V=christoffel_x_vector(V,metric_einsteinpy,sp_symbols, vector_is_covariant)


    if vector_is_covariant:

        return partial_matrix-ch_V

    else:

        return partial_matrix+ch_V


    
    

In [47]:
# Define the coordinates for 2D polar coordinates (r, theta)
r, theta = sp.symbols('r theta')


metric = [[0 for i in range(2)] for i in range(2)]
metric[0][0] = 1
metric[1][1] = r**2

# Define the symbols array
symbols = [r, theta]

# Create a MetricTensor object
m_obj = MetricTensor(metric, symbols)



V_test = sp.Matrix([r,theta])

test_contravariant=covariant_derivative(V_test,m_obj,symbols, vector_is_covariant=False)

V_test = sp.Matrix([1,2])

test_covariant=covariant_derivative(V_test,m_obj,symbols, vector_is_covariant=True)


display(test_contravariant)

display(test_covariant)

Matrix([
[       1, theta/r],
[-r*theta,       2]])

Matrix([
[   0, -2/r],
[-2/r,    r]])

## Operating System

### Functions

In [48]:
def get_tetrahed(user_input_metric):
    #The tetrahed is defined as in https://arxiv.org/abs/2404.03095
 
    # Define the scalars A, B, C, and D
    # Define the scalars with adjusted indices
    A = g[9]
    
    B = -g[8]**2 + g[7] * g[9]
    
    C = (-g[4] * g[8]**2 - g[7] * g[6]**2 - g[9] * g[5]**2
         + 2 * g[5] * g[6] * g[8] + g[4] * g[7] * g[9])
    
    
    D = (g[1]**2 * g[8]**2 + g[2]**2 * g[6]**2 + g[3]**2 * g[5]**2
         - g[7] * g[9] * g[1]**2 - g[4] * g[9] * g[2]**2 - g[4] * g[7] * g[3]**2
         - g[0] * g[9] * g[5]**2 - g[0] * g[7] * g[6]**2 - g[0] * g[4] * g[8]**2
         + 2 * g[9] * g[1] * g[2] * g[5] + 2 * g[7] * g[1] * g[3] * g[6]
         + 2 * g[4] * g[2] * g[3] * g[8] + 2 * g[0] * g[5] * g[6] * g[8] 
         - 2 * g[1] * g[2] * g[6] * g[8] - 2 * g[1] * g[3] * g[5] * g[8]
         - 2 * g[2] * g[3] * g[5] * g[6] + g[0] * g[4] * g[7] * g[9])
    
    
    #note : (A,B,C,D) = (-,+,+,+)
    
    E_0 =(1 / sp.sqrt(-C * D)) * sp.Matrix([
        C,   #here, first error of the paper, it's a +C not -C             
         
         g[1] * g[8]**2 + g[2] * g[5] * g[9] + g[3] * g[6] * g[7] - g[1] * g[7] * g[9] - g[2] * g[6] * g[8] - g[3] * g[5] * g[8],
    
        g[2] * g[6]**2 + g[1] * g[5] * g[9] + g[3] * g[4] * g[8] - g[1] * g[6] * g[8] - g[2] * g[4] * g[9] - g[3] * g[5] * g[6],
    
        g[3] * g[5]**2 + g[1] * g[6] * g[7] + g[2] * g[4] * g[8] - g[1] * g[5] * g[8] - g[2] * g[5] * g[6] - g[3] * g[4] * g[7]
    
    ])
    
    E_1=(1 / sp.sqrt(B*C)) * sp.Matrix([
        0,                
         
        B,
    
        #g[5] * g[9] - g[6] * g[8], second error in the paper
        -g[5] * g[9] + g[6] * g[8],
    
        g[5] * g[8] - g[6] * g[7]
    
    ])
    
    E_2=(1 / sp.sqrt(A*B)) * sp.Matrix([
        0,                
         
        0,
    
        A,
    
        -g[8] 
    
    ])
    
    E_3=(1 / sp.sqrt(A)) * sp.Matrix([
        0,                
         
        0,
    
        0,
    
        1
    
    ])
    
    E_solution=matrix = sp.Matrix.hstack(E_0, E_1, E_2, E_3)
    
    
    
    E_0=sp.simplify(E_0.subs(user_input_metric))
    E_1=sp.simplify(E_1.subs(user_input_metric))
    E_2=sp.simplify(E_2.subs(user_input_metric))
    E_3=sp.simplify(E_3.subs(user_input_metric))
    
    E_solution=sp.simplify(E_solution.subs(user_input_metric))

    return E_0, E_1, E_2, E_3, E_solution
        

In [112]:
class Warp_drive:
    def __init__(self, sp_symbols, function_substitutions, metric_from_user_input):
        """
        Initializes a new Warp_drive object.
        
        :sp_symbols: List of symbols to represent spacetime coordinate.
        
        :function_substitutions. Dictionnary of sympy functions
            example: {f:f_actual,v:v_actual}
            f and v are the symbolic function depending on spacetime coordinates and f_actual 
            and v_actual are the actual functions expression.
            
        :metric_from_user_input: Dictionnary associating g1,g2,...,g9 the symbol of the metric 
        to the desired metric.
        
        """
        self.symbols = sp_symbols
        self.f_subs = function_substitutions
        self.user_input_metric = metric_from_user_input


    def build_and_compute(self):
       
    
        #Construct the template on which we are going to substitute user inputs
        metric_symbol = sp.symbols('g1:11')
    
        metric_template = sp.Matrix([
            [g[0], g[1], g[2], g[3]],
            [g[1], g[4], g[5], g[6]],
            [g[2], g[5], g[7], g[8]],
            [g[3], g[6], g[8], g[9]]
        ])
    
        
    
        # Update the template
        Metric = metric_template.subs(self.user_input_metric)
        
        # Create a MetricTensor object that can be used with einsteinpy
        m_obj = MetricTensor(sp.Array(Metric), self.symbols)
    
        #no need to compute the following quantity but they are available at this point
        
        # Compute Christoffel symbols
        #ch = ChristoffelSymbols.from_metric(m_obj)
        
        # Compute Riemann Curvature Tensor
        #riemann = RiemannCurvatureTensor.from_christoffels(ch)
        
        # Compute Ricci Tensor
        #ricci = RicciTensor.from_riemann(riemann)
        
        # Compute Ricci Scalar
        #ricci_scalar = RicciScalar.from_riccitensor(ricci)
        
        # Compute EinsteinTensor
        einst = EinsteinTensor.from_metric(m_obj)
        #we go back to sympy object
        Einstein_tensor=einst.tensor() 
        Tuv=sp.simplify(sp.Matrix(Einstein_tensor)) #mind that the unit are not yet correct
    
        self.Metric=Metric
        self.Tuv=Tuv
    
        print("Einstein tensor computed.")#############################################################################
    
    
        #Get tetrahed
        E_0, E_1, E_2, E_3, E_solution = get_tetrahed(self.user_input_metric)
    
        # The tetrahed corresponding to the local observer whose 4-velocity vector is orthogonal 
        # to constant time hypersurface. 
    
        print("Tetrahed computed.")####################################################################################
        
    
        #compute Tu^v^, the energy-momentum tensor from the perspective of a local observer 
        Tuv_ref=sp.simplify((E_solution.T)*Tuv*E_solution)
    
        self.Tuv_ref=Tuv_ref
        
        #NEC ====================================================================
        
        # Define symbolic variables theta and psi
        theta, psi = sp.symbols('theta psi')
        
        # Define the parametric null-vector (1, sin(theta)*cos(psi), sin(theta)*sin(psi), cos(theta)) 
        ku = sp.Matrix([
            1,
            sp.sin(theta) * sp.cos(psi),
            sp.sin(theta) * sp.sin(psi),
            sp.cos(theta)
        ])
        
        NEC=ku.T*Tuv_ref*ku
        NEC=NEC[0,0] #conversion from matrix (1,1) to scalar
    
        self.NEC=NEC
        
        #WEC ====================================================================
        
        # Define the rapidities
        eta_1, eta_2, eta_3 = sp.symbols('eta_1 eta_2 eta_3')
        
        # Define the hyperbolic functions
        cosh_1, sinh_1 = sp.cosh(eta_1), sp.sinh(eta_1)
        cosh_2, sinh_2 = sp.cosh(eta_2), sp.sinh(eta_2)
        cosh_3, sinh_3 = sp.cosh(eta_3), sp.sinh(eta_3)
        
        # Lorentz boost in the x-direction (eta_1)
        Lambda_1 = sp.Matrix([
            [cosh_1, -sinh_1, 0, 0],
            [-sinh_1, cosh_1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
        
        # Lorentz boost in the y-direction (eta_2)
        Lambda_2 = sp.Matrix([
            [cosh_2, 0, -sinh_2, 0],
            [0, 1, 0, 0],
            [-sinh_2, 0, cosh_2, 0],
            [0, 0, 0, 1]
        ])
        
        # Lorentz boost in the z-direction (eta_3)
        Lambda_3 = sp.Matrix([
            [cosh_3, 0, 0, -sinh_3],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [-sinh_3, 0, 0, cosh_3]
        ])
        
        # Define the parametric time-vector
        Uu=sp.Matrix([1,0,0,0])
        Uu=Lambda_1*Lambda_2*Lambda_3*Uu
        
        WEC=Uu.T*Tuv_ref*Uu
        WEC=WEC[0,0]
    
        self.WEC=WEC
        
        #SEC ====================================================================
        minkowski=sp.Matrix([
            [-1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
        
        
        trace_Tuv_ref=-Tuv_ref[0,0]+Tuv_ref[1,1]+Tuv_ref[2,2]+Tuv_ref[3,3]
        
        SEC=Uu.T*(Tuv_ref-0.5*(trace_Tuv_ref)*minkowski)*Uu
        SEC=SEC[0,0]
    
        self.SEC=SEC
        
        #DEC ====================================================================
        
        minkowski_inv=minkowski
        
        energy_flow=-minkowski_inv*Tuv_ref*Uu
    
        #the original dominant energy condition
        DEC_intern=energy_flow.T*minkowski*energy_flow
        DEC_intern=DEC_intern[0,0]
    
        #we define the dominant energy condition as in https://arxiv.org/abs/2404.03095 to make it comparable to other energy conditions
        DEC= -sp.sign(DEC_intern) * sp.sqrt(sp.Abs(DEC_intern))
    
        self.DEC=DEC
        
        print("Energy conditions computed.")###########################################################################
        
    
        #note that in this section, the quantities computed are not expressed in the standard units yet
    
        
        #Momentum ===============================================================
        momentum=Tuv_ref[1:,0]
        
        self.Momentum=momentum
        
        #Metric scalar ==========================================================
    
        #4-velocity vector
        Uu=E_0
        
        U_u=Metric*Uu
        
        Metric_inv=Metric.inv()
        
        P_uv = Metric + U_u*U_u.T
    
        Du_Uv=covariant_derivative(U_u,m_obj,self.symbols, vector_is_covariant=True)
    
    
        
        Pa_u = sp.zeros(4, 4)
        
        for a in range(4):
            for u in range(4):
                Pa_u[a,u] = sum(Metric_inv[a,b] * P_uv[b,u] for b in range(4))
        Pb_v = Pa_u.copy()
        
    
        
        A_av = sp.zeros(4, 4)
    
        for a in range(4):
            for v in range(4):
                A_av[a, v] = sum(Pb_v[b,v] * Du_Uv[a,b] for b in range(4))
    
    
        
        B_uv = sp.zeros(4, 4)
    
        for u in range(4):
            for v in range(4):
                B_uv[u,v] = sum(Pa_u[a,u] * A_av[a,v] for a in range(4))
    
        
    
    
        expansion_uv=B_uv+B_uv.T
        expansion_scalar=sp.simplify(sum(Metric_inv[u,v] * expansion_uv[u,v] for u in range(4) for v in range(4)))
        self.expansion_scalar=expansion_scalar
    
        
        shear_uv=expansion_uv-expansion_scalar*P_uv/3
        shear_uv_up = Metric_inv * shear_uv * Metric_inv
        shear_scalar = sp.simplify(sp.sqrt(1/2*sp.simplify(sum(shear_uv[u,v] * shear_uv_up[u,v] for u in range(4) for v in range(4)))))
        self.shear_scalar=shear_scalar
    
    
        
        vorticity_uv=B_uv-B_uv.T
        vorticity_uv_up = Metric_inv * vorticity_uv * Metric_inv
        vorticity_scalar = sp.simplify(sp.sqrt(1/2*sp.simplify(sum(vorticity_uv[u,v] * vorticity_uv_up[u,v] for u in range(4) for v in range(4)))))
        self.vorticity_scalar=vorticity_scalar
        
    
        
        print("Metric scalar computed.")###########################################################################
    
    
        #Extrinsic Curvature
    
    
        #convert metric to 3+1 formalism
        #first we need to establish what alpha beta gamma are in terms of g1,g2,g3,...
        
        # Define the 3+1 variables
        alpha, beta1, beta2, beta3 = sp.symbols('alpha beta1 beta2 beta3')
        gamma11, gamma12, gamma13 = sp.symbols('gamma11 gamma12 gamma13')
        gamma22, gamma23 = sp.symbols('gamma22 gamma23')
        gamma33 = sp.symbols('gamma33')
        
        Gamma=sp.Matrix([
            [gamma11, gamma12, gamma13],
            [gamma12, gamma22, gamma23],
            [gamma13, gamma23, gamma33]
        ])
        
        beta_vector = sp.Matrix([beta1, beta2, beta3])
        
        
        # Compute the covariant components beta_i
        beta_covariant = sp.simplify(Gamma.inv()) * beta_vector
        
        # Compute beta^i * beta_i
        beta_dot_beta = sp.simplify(beta_vector.dot(beta_covariant))
        
        
        
        # Those several substitutions allow to obtain alpha beta gamma are in terms of g1,g2,g3,...
        substitution_step1= {
            beta1:g[1],
            beta2:g[2],
            beta3:g[3],
            gamma11:g[4],
            gamma12:g[5],
            gamma13:g[6],
            gamma22:g[7],
            gamma23:g[8],
            gamma33:g[9]
        }
        
        substitution_step2= {
            alpha:sp.sqrt(-g[0]+beta_dot_beta.subs(substitution_step1))
        }
        
        substitution_3_1_form= substitution_step1 | substitution_step2
        
        
        
        #compute Kij and its scalar
        symbols_3=self.symbols[1:] #we take only spatial coordinate into accounts to compute Kij
        
        
        metric_3=Gamma.subs(substitution_3_1_form).subs(self.user_input_metric)
        metric_3_einsteinpy = MetricTensor(sp.Array(metric_3), symbols_3)
        
        DiBj=covariant_derivative(beta_vector.subs(substitution_3_1_form).subs(self.user_input_metric),metric_3_einsteinpy,symbols_3, vector_is_covariant=True)
        
        #Kij=(1/2α) * (∇ i βj + ∇ j βi - ∂/∂t γij )
        Kij=sp.simplify(1/(2*alpha.subs(substitution_3_1_form).subs(self.user_input_metric))*(DiBj+DiBj.T-sp.diff(metric_3,self.symbols[0])))
        
        K=sp.simplify(-alpha.subs(substitution_3_1_form).subs(self.user_input_metric)*sum(metric_3.inv()[i,j]* Kij[i,j] for i in range(3) for j in range(3)))
    
    
        self.alpha=sp.simplify(alpha.subs(substitution_3_1_form).subs(self.user_input_metric))
        self.beta1=beta1.subs(substitution_3_1_form).subs(self.user_input_metric)
        self.beta2=beta2.subs(substitution_3_1_form).subs(self.user_input_metric)
        self.beta3=beta3.subs(substitution_3_1_form).subs(self.user_input_metric)
        self.beta_beta=beta_dot_beta.subs(substitution_3_1_form).subs(self.user_input_metric)
        self.Gamma=Gamma.subs(substitution_3_1_form).subs(self.user_input_metric)
    
        self.K=K
        
        
    
        print("Extrinsic curvature computed.")###########################################################################
    
        

    def greet(self):
        """
        Prints a greeting message 
        """
        print(f"Hello, this is a test.")

   

In [113]:
#Numerical Evaluation


#to display

#beta1,2,3 [unit = 1]
#gamma matrix [unit = 1]
#alpha [unit = 1]
#T^^ for n0 (conversion needed) [unit = J m^-3]
#T^^ for minimum values among all observer (conversion needed) [unit = J m^-3]
#stress scalar (might be the same as expansion scalar) [unit = J m^-3]
#shear scalar [unit = m^-1]
#vorticity scalar [unit = m^-1]
#K scalar [unit = m^-1]
#Momentum flow (conversion needed with extra /c to go back to momentum)  [unit = kg m s^-1 m^-3]
#expansion scalar [unit = m^-1]
#NEC (conversion needed) [unit = J m^-3]
#WEC (conversion needed) [unit = J m^-3]
#SEC (conversion needed) [unit = J m^-3]
#DEC (conversion needed) [unit = J m^-3]



gravity_const=6.6743*1e-11
c_speed_const=299792458
pi_8=8*np.pi
physical_energy_conversion=c_speed_const**4/pi_8/gravity_const
physical_momentum_conversion=physical_energy_conversion/c_speed_const

#additional notes:
#   time must be given in distance (thus multiply by c).
#   R = 1/m^2
#   g=1
#   Tuv = c^4/8piG = J/m^3


    
    
    
    


   

    

### Workflow 

In [114]:
##################################################### USER INPUT ########################################################


# Define substitutions for the new variables

t,x,y,z = sp.symbols('t x y z')
symbols = [t,x,y,z] #IMPORTANT first symbol must be time

# Define the functions to be input in the metric
v = sp.Function('v')(t)
f = sp.Function('f')(t, x, y, z)

f_subs={}

#user input metric using symbol
user_input_metric = {
    g[0]: -1 + v**2*f**2,
    g[1]: -v*f,
    g[2]: 0,
    g[3]: 0,
    g[4]: 1,
    g[5]: 0,
    g[6]: 0,
    g[7]: 1,
    g[8]: 0,
    g[9]: 1   
}

##########################################################################################################################


warp_test=Warp_drive(symbols, f_subs, user_input_metric)
warp_test.build_and_compute()

Einstein tensor computed.
Tetrahed computed.
Energy conditions computed.
Metric scalar computed.
Extrinsic curvature computed.


In [115]:
display(warp_test.Metric)

Matrix([
[f(t, x, y, z)**2*v(t)**2 - 1, -f(t, x, y, z)*v(t), 0, 0],
[         -f(t, x, y, z)*v(t),                   1, 0, 0],
[                           0,                   0, 1, 0],
[                           0,                   0, 0, 1]])

In [116]:
display(warp_test.Tuv)

Matrix([
[(-0.75*f(t, x, y, z)**2*v(t)**2*Derivative(f(t, x, y, z), y)**2 - 0.75*f(t, x, y, z)**2*v(t)**2*Derivative(f(t, x, y, z), z)**2 - 1.0*f(t, x, y, z)*Derivative(f(t, x, y, z), (y, 2)) - 1.0*f(t, x, y, z)*Derivative(f(t, x, y, z), (z, 2)) - 0.25*Derivative(f(t, x, y, z), y)**2 - 0.25*Derivative(f(t, x, y, z), z)**2)*v(t)**2,            (0.75*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), y)**2 + 0.75*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), z)**2 + 0.5*Derivative(f(t, x, y, z), (y, 2)) + 0.5*Derivative(f(t, x, y, z), (z, 2)))*v(t), -(f(t, x, y, z)**2*v(t)**2*Derivative(f(t, x, y, z), x, y) + 2*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x)*Derivative(f(t, x, y, z), y) + f(t, x, y, z)*v(t)*Derivative(f(t, x, y, z), t, y) + f(t, x, y, z)*Derivative(f(t, x, y, z), y)*Derivative(v(t), t) + Derivative(f(t, x, y, z), x, y))*v(t)/2, -(f(t, x, y, z)**2*v(t)**2*Derivative(f(t, x, y, z), x, z) + 2*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x)*Derivative(f(t, x, y, z), z) 

In [117]:
display(warp_test.Tuv_ref)

Matrix([
[0.25*(-Derivative(f(t, x, y, z), y)**2 - Derivative(f(t, x, y, z), z)**2)*v(t)**2,                                                                                                                                          0.5*(Derivative(f(t, x, y, z), (y, 2)) + Derivative(f(t, x, y, z), (z, 2)))*v(t),                                                                                                                                                                                                                                                                    -v(t)*Derivative(f(t, x, y, z), x, y)/2,                                                                                                                                                                                                                                                                    -v(t)*Derivative(f(t, x, y, z), x, z)/2],
[ 0.5*(Derivative(f(t, x, y, z), (y, 2)) + Derivative(f(t, x, y, z), (z, 2)))*v(t),    

In [118]:
display(warp_test.NEC)

0.25*(-Derivative(f(t, x, y, z), y)**2 - Derivative(f(t, x, y, z), z)**2)*v(t)**2 + 0.5*(Derivative(f(t, x, y, z), (y, 2)) + Derivative(f(t, x, y, z), (z, 2)))*v(t)*sin(theta)*cos(psi) + ((f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x, y)/2 + v(t)**2*Derivative(f(t, x, y, z), x)*Derivative(f(t, x, y, z), y) + v(t)*Derivative(f(t, x, y, z), t, y)/2 + Derivative(f(t, x, y, z), y)*Derivative(v(t), t)/2)*sin(theta)*cos(psi) + (-1.0*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), (x, 2)) - 1.0*v(t)**2*Derivative(f(t, x, y, z), x)**2 + 0.25*v(t)**2*Derivative(f(t, x, y, z), y)**2 - 0.25*v(t)**2*Derivative(f(t, x, y, z), z)**2 - 1.0*v(t)*Derivative(f(t, x, y, z), t, x) - 1.0*Derivative(f(t, x, y, z), x)*Derivative(v(t), t))*sin(psi)*sin(theta) + v(t)**2*cos(theta)*Derivative(f(t, x, y, z), y)*Derivative(f(t, x, y, z), z)/2 - v(t)*Derivative(f(t, x, y, z), x, y)/2)*sin(psi)*sin(theta) + ((f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x, z)/2 + v(t)**2*Derivative(f(t, x, y, z), x)*Derivat

In [119]:
display(warp_test.WEC)

-(-(f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x, y)/2 + v(t)**2*Derivative(f(t, x, y, z), x)*Derivative(f(t, x, y, z), y) + v(t)*Derivative(f(t, x, y, z), t, y)/2 + Derivative(f(t, x, y, z), y)*Derivative(v(t), t)/2)*sinh(eta_1)*cosh(eta_2)*cosh(eta_3) - (-1.0*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), (x, 2)) - 1.0*v(t)**2*Derivative(f(t, x, y, z), x)**2 + 0.25*v(t)**2*Derivative(f(t, x, y, z), y)**2 - 0.25*v(t)**2*Derivative(f(t, x, y, z), z)**2 - 1.0*v(t)*Derivative(f(t, x, y, z), t, x) - 1.0*Derivative(f(t, x, y, z), x)*Derivative(v(t), t))*sinh(eta_2)*cosh(eta_3) - v(t)**2*sinh(eta_3)*Derivative(f(t, x, y, z), y)*Derivative(f(t, x, y, z), z)/2 - v(t)*cosh(eta_1)*cosh(eta_2)*cosh(eta_3)*Derivative(f(t, x, y, z), x, y)/2)*sinh(eta_2)*cosh(eta_3) - (-(f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x, z)/2 + v(t)**2*Derivative(f(t, x, y, z), x)*Derivative(f(t, x, y, z), z) + v(t)*Derivative(f(t, x, y, z), t, z)/2 + Derivative(f(t, x, y, z), z)*Derivative(v(t), t)/2)*sinh(e

In [120]:
display(warp_test.SEC)

-(-(-0.25*(-Derivative(f(t, x, y, z), y)**2 - Derivative(f(t, x, y, z), z)**2)*v(t)**2 - 0.25*v(t)**2*Derivative(f(t, x, y, z), y)**2 + 0.25*v(t)**2*Derivative(f(t, x, y, z), z)**2)*sinh(eta_3) - (f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x, z)/2 + v(t)**2*Derivative(f(t, x, y, z), x)*Derivative(f(t, x, y, z), z) + v(t)*Derivative(f(t, x, y, z), t, z)/2 + Derivative(f(t, x, y, z), z)*Derivative(v(t), t)/2)*sinh(eta_1)*cosh(eta_2)*cosh(eta_3) - v(t)**2*sinh(eta_2)*cosh(eta_3)*Derivative(f(t, x, y, z), y)*Derivative(f(t, x, y, z), z)/2 - v(t)*cosh(eta_1)*cosh(eta_2)*cosh(eta_3)*Derivative(f(t, x, y, z), x, z)/2)*sinh(eta_3) - (-(-0.25*(-Derivative(f(t, x, y, z), y)**2 - Derivative(f(t, x, y, z), z)**2)*v(t)**2 + 0.25*v(t)**2*Derivative(f(t, x, y, z), y)**2 - 0.25*v(t)**2*Derivative(f(t, x, y, z), z)**2)*sinh(eta_2)*cosh(eta_3) - (f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x, y)/2 + v(t)**2*Derivative(f(t, x, y, z), x)*Derivative(f(t, x, y, z), y) + v(t)*Derivative(f(t, x, y, z

In [121]:
display(warp_test.DEC)

-sqrt(Abs((-(-f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x, y)/2 - v(t)**2*Derivative(f(t, x, y, z), x)*Derivative(f(t, x, y, z), y) - v(t)*Derivative(f(t, x, y, z), t, y)/2 - Derivative(f(t, x, y, z), y)*Derivative(v(t), t)/2)*sinh(eta_1)*cosh(eta_2)*cosh(eta_3) - (1.0*f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), (x, 2)) + 1.0*v(t)**2*Derivative(f(t, x, y, z), x)**2 - 0.25*v(t)**2*Derivative(f(t, x, y, z), y)**2 + 0.25*v(t)**2*Derivative(f(t, x, y, z), z)**2 + 1.0*v(t)*Derivative(f(t, x, y, z), t, x) + 1.0*Derivative(f(t, x, y, z), x)*Derivative(v(t), t))*sinh(eta_2)*cosh(eta_3) + v(t)**2*sinh(eta_3)*Derivative(f(t, x, y, z), y)*Derivative(f(t, x, y, z), z)/2 + v(t)*cosh(eta_1)*cosh(eta_2)*cosh(eta_3)*Derivative(f(t, x, y, z), x, y)/2)**2 + (-(-f(t, x, y, z)*v(t)**2*Derivative(f(t, x, y, z), x, z)/2 - v(t)**2*Derivative(f(t, x, y, z), x)*Derivative(f(t, x, y, z), z) - v(t)*Derivative(f(t, x, y, z), t, z)/2 - Derivative(f(t, x, y, z), z)*Derivative(v(t), t)/2)*sinh(eta_1)*cosh(

In [122]:
display(warp_test.Momentum)

Matrix([
[0.5*(Derivative(f(t, x, y, z), (y, 2)) + Derivative(f(t, x, y, z), (z, 2)))*v(t)],
[                                         -v(t)*Derivative(f(t, x, y, z), x, y)/2],
[                                         -v(t)*Derivative(f(t, x, y, z), x, z)/2]])

In [123]:
display(warp_test.expansion_scalar)

2*v(t)*Derivative(f(t, x, y, z), x)

In [124]:
display(warp_test.shear_scalar)

0.333333333333333*sqrt(3)*sqrt((4*Derivative(f(t, x, y, z), x)**2 + 3*Derivative(f(t, x, y, z), y)**2 + 3*Derivative(f(t, x, y, z), z)**2)*v(t)**2)

In [125]:
display(warp_test.vorticity_scalar)

0

In [126]:
display(warp_test.alpha)

1

In [127]:
display(warp_test.beta1)

-f(t, x, y, z)*v(t)

In [128]:
display(warp_test.beta2)

0

In [129]:
display(warp_test.beta3)

0

In [130]:
display(warp_test.beta_beta)

f(t, x, y, z)**2*v(t)**2

In [131]:
display(warp_test.Gamma)

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [132]:
display(warp_test.K)

v(t)*Derivative(f(t, x, y, z), x)

In [81]:
    #Numerical Evaluation


    #to display
    
    #beta1,2,3 [unit = 1]
    #gamma matrix [unit = 1]
    #alpha [unit = 1]
    #T^^ for n0 (conversion needed) [unit = J m^-3]
    #T^^ for minimum values among all observer (conversion needed) [unit = J m^-3]
    #stress scalar (might be the same as expansion scalar) [unit = J m^-3]
    #shear scalar [unit = m^-1]
    #vorticity scalar [unit = m^-1]
    #K scalar [unit = m^-1]
    #Momentum flow (conversion needed with extra /c to go back to momentum)  [unit = kg m s^-1 m^-3]
    #expansion scalar [unit = m^-1]
    #NEC (conversion needed) [unit = J m^-3]
    #WEC (conversion needed) [unit = J m^-3]
    #SEC (conversion needed) [unit = J m^-3]
    #DEC (conversion needed) [unit = J m^-3]
    
    
    
    gravity_const=6.6743*1e-11
    c_speed_const=299792458
    pi_8=8*np.pi
    physical_energy_conversion=c_speed_const**4/pi_8/gravity_const
    physical_momentum_conversion=physical_energy_conversion/c_speed_const

    #additional notes:
    #   time must be given in distance (thus multiply by c).
    #   R = 1/m^2
    #   g=1
    #   Tuv = c^4/8piG = J/m^3
    
